### Technical Resources - Python Jupyter and SQLite3 (Python package for Database)

# Step 1: Create database and table using SQLite to store the football data 

In [302]:
## Python Package Used

import sqlite3
from pandas.io.json import json_normalize
import pandas as pd 
import datetime
import http.client
import json
import http.client

### Setup database connectio/ SQLite 

In [303]:
conn = sqlite3.connect('test.db')   
print(conn)

### Create Football Data Table 

In [317]:
conn.execute('drop table if exists football') # we can also drop the table with this

In [318]:
# Create the Football Data Table 
conn.execute('''CREATE TABLE football

    (id          INT PRIMARY KEY NOT NULL,
    div         VARCHAR       NOT NULL,
    date_time   TIMESTAMP     NOT NULL,
    home_team   VARCHAR       NOT NULL,
    away_team   VARCHAR       NOT NULL,
    fthg        INT           DEFAULT(0),
    ftag        INT           DEFAULT(0));''')

#### Test Table Exist 

In [319]:
conn.execute("SELECT * from football") # check if the table exist 

# Step 2:  Connect to Football data website to retrieve the data 

In [320]:
# Retrive Football data from https://www.football-data.co.uk

connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': 'f403d7fd891241aebfb853bc39f7f234' }
connection.request('GET', '/v2/matches?competitions=PL,PL&status=FINISHED&dateFrom=2019-12-26&dateTo=2020-01-05', None, headers )
#connection.request('GET', '/v2/competitions/2021/matches', None, headers )

response = json.loads(connection.getresponse().read().decode())


In [321]:
response

{'count': 30,
 'filters': {'competitions': [2021],
  'dateFrom': '2019-12-26',
  'dateTo': '2020-01-05',
  'permission': 'TIER_ONE',
  'status': ['FINISHED']},
 'matches': [{'awayTeam': {'id': 397, 'name': 'Brighton & Hove Albion FC'},
   'competition': {'id': 2021, 'name': 'Premier League'},
   'group': 'Regular Season',
   'homeTeam': {'id': 73, 'name': 'Tottenham Hotspur FC'},
   'id': 264524,
   'lastUpdated': '2019-12-26T23:59:24Z',
   'matchday': 19,
   'referees': [{'id': 11503, 'name': 'Graham Scott', 'nationality': None},
    {'id': 11424, 'name': 'Neil Davies', 'nationality': None},
    {'id': 137388, 'name': None, 'nationality': None},
    {'id': 11396, 'name': 'Tim Robinson', 'nationality': None},
    {'id': 11575, 'name': 'Mike Dean', 'nationality': None},
    {'id': 11521, 'name': 'Mark Scholes', 'nationality': None}],
   'score': {'duration': 'REGULAR',
    'extraTime': {'awayTeam': None, 'homeTeam': None},
    'fullTime': {'awayTeam': 1, 'homeTeam': 2},
    'halfTime': 

#### Normalised the football json data and transform into Pandas dataframe

In [325]:
Football_data = json_normalize(response['matches']) 
Football_data.head(5)

,awayTeam.id,awayTeam.name,competition.id,competition.name,group,homeTeam.id,homeTeam.name,id,lastUpdated,matchday,...,score.penalties.homeTeam,score.winner,season.currentMatchday,season.endDate,season.id,season.startDate,season.winner,stage,status,utcDate
0,397,Brighton & Hove Albion FC,2021,Premier League,Regular Season,73,Tottenham Hotspur FC,264524,2019-12-26T23:59:24Z,19,...,None,HOME_TEAM,23,2020-05-17,468,2019-08-09,None,REGULAR_SEASON,FINISHED,2019-12-26T12:30:00Z
1,68,Norwich City FC,2021,Premier League,Regular Season,58,Aston Villa FC,264522,2019-12-26T23:59:24Z,19,...,None,HOME_TEAM,23,2020-05-17,468,2019-08-09,None,REGULAR_SEASON,FINISHED,2019-12-26T15:00:00Z
2,340,Southampton FC,2021,Premier League,Regular Season,61,Chelsea FC,264523,2019-12-26T23:59:24Z,19,...,None,AWAY_TEAM,23,2020-05-17,468,2019-08-09,None,REGULAR_SEASON,FINISHED,2019-12-26T15:00:00Z
3,328,Burnley FC,2021,Premier League,Regular Season,62,Everton FC,264525,2019-12-26T23:59:24Z,19,...,None,HOME_TEAM,23,2020-05-17,468,2019-08-09,None,REGULAR_SEASON,FINISHED,2019-12-26T15:00:00Z
4,346,Watford FC,2021,Premier League,Regular Season,356,Sheffield United FC,264527,2019-12-26T23:59:24Z,19,...,None,DRAW,23,2020-05-17,468,2019-08-09,None,REGULAR_SEASON,FINISHED,2019-12-26T15:00:00Z


###  Data Processing 
###### 1. Select  columns of interest
###### 2. Format date to timestamp
###### 3. Rename coloum to match database table


In [326]:

Football_data['utcDate'] = pd.to_datetime(Football_data['utcDate'])
Football_data1= Football_data.rename(columns={'utcDate': 'date_time', 'competition.name': 'div', 'homeTeam.name': 'home_team', 
'awayTeam.name': 'away_team', 'score.fullTime.homeTeam':'fthg','score.fullTime.awayTeam':'ftag'})
Football_data1= Football_data.rename(columns={'utcDate': 'date_time', 'competition.name': 'div','homeTeam.name': 'home_team', 'awayTeam.name': 'away_team', 'score.fullTime.homeTeam':'fthg','score.fullTime.awayTeam':'ftag'})
Football_data2=Football_data1[['id','date_time', 'div', 'away_team', 'home_team','ftag','fthg']]
Football_data2=Football_data2.set_index('id')
Football_data2.columns
Football_data2

,date_time,div,away_team,home_team,ftag,fthg
id,,,,,,
264524,2019-12-26 12:30:00,Premier League,Brighton & Hove Albion FC,Tottenham Hotspur FC,1,2
264522,2019-12-26 15:00:00,Premier League,Norwich City FC,Aston Villa FC,0,1
264523,2019-12-26 15:00:00,Premier League,Southampton FC,Chelsea FC,2,0
264525,2019-12-26 15:00:00,Premier League,Burnley FC,Everton FC,0,1
264527,2019-12-26 15:00:00,Premier League,Watford FC,Sheffield United FC,1,1
264529,2019-12-26 15:00:00,Premier League,West Ham United FC,Crystal Palace FC,1,2
264530,2019-12-26 15:00:00,Premier League,Arsenal FC,AFC Bournemouth,1,1
264526,2019-12-26 17:30:00,Premier League,Newcastle United FC,Manchester United FC,1,4
264528,2019-12-26 20:00:00,Premier League,Liverpool FC,Leicester City FC,4,0


#### Insert Football Data into Football Table in Database

In [327]:
Football_data2.to_sql('football', con=conn, if_exists='append')

#### Result 

In [328]:
for row in conn.execute('SELECT * FROM football'):
        print (row)

(264524, 'Premier League', '2019-12-26 12:30:00', 'Tottenham Hotspur FC', 'Brighton & Hove Albion FC', 2, 1)
(264522, 'Premier League', '2019-12-26 15:00:00', 'Aston Villa FC', 'Norwich City FC', 1, 0)
(264523, 'Premier League', '2019-12-26 15:00:00', 'Chelsea FC', 'Southampton FC', 0, 2)
(264525, 'Premier League', '2019-12-26 15:00:00', 'Everton FC', 'Burnley FC', 1, 0)
(264527, 'Premier League', '2019-12-26 15:00:00', 'Sheffield United FC', 'Watford FC', 1, 1)
(264529, 'Premier League', '2019-12-26 15:00:00', 'Crystal Palace FC', 'West Ham United FC', 2, 1)
(264530, 'Premier League', '2019-12-26 15:00:00', 'AFC Bournemouth', 'Arsenal FC', 1, 1)
(264526, 'Premier League', '2019-12-26 17:30:00', 'Manchester United FC', 'Newcastle United FC', 4, 1)
(264528, 'Premier League', '2019-12-26 20:00:00', 'Leicester City FC', 'Liverpool FC', 0, 4)
(264521, 'Premier League', '2019-12-27 19:45:00', 'Wolverhampton Wanderers FC', 'Manchester City FC', 3, 2)
(264539, 'Premier League', '2019-12-28 12

#### Result as Pandas Dataframe

In [339]:
df = pd.read_sql_query("SELECT * FROM football Limte ;", conn)
df

,id,div,date_time,home_team,away_team,fthg,ftag
0,264524,Premier League,2019-12-26 12:30:00,Tottenham Hotspur FC,Brighton & Hove Albion FC,2,1
1,264522,Premier League,2019-12-26 15:00:00,Aston Villa FC,Norwich City FC,1,0
2,264523,Premier League,2019-12-26 15:00:00,Chelsea FC,Southampton FC,0,2
3,264525,Premier League,2019-12-26 15:00:00,Everton FC,Burnley FC,1,0
4,264527,Premier League,2019-12-26 15:00:00,Sheffield United FC,Watford FC,1,1
5,264529,Premier League,2019-12-26 15:00:00,Crystal Palace FC,West Ham United FC,2,1
6,264530,Premier League,2019-12-26 15:00:00,AFC Bournemouth,Arsenal FC,1,1
7,264526,Premier League,2019-12-26 17:30:00,Manchester United FC,Newcastle United FC,4,1
8,264528,Premier League,2019-12-26 20:00:00,Leicester City FC,Liverpool FC,0,4
9,264521,Premier League,2019-12-27 19:45:00,Wolverhampton Wanderers FC,Manchester City FC,3,2


In [345]:
pd.read_sql_query("Select * from football where date_time <'2020-01-01 12:30:00' and fthg!= 0 and ftag != 0  ;", conn)

,id,div,date_time,home_team,away_team,fthg,ftag
0,264524,Premier League,2019-12-26 12:30:00,Tottenham Hotspur FC,Brighton & Hove Albion FC,2,1
1,264527,Premier League,2019-12-26 15:00:00,Sheffield United FC,Watford FC,1,1
2,264529,Premier League,2019-12-26 15:00:00,Crystal Palace FC,West Ham United FC,2,1
3,264530,Premier League,2019-12-26 15:00:00,AFC Bournemouth,Arsenal FC,1,1
4,264526,Premier League,2019-12-26 17:30:00,Manchester United FC,Newcastle United FC,4,1
5,264521,Premier League,2019-12-27 19:45:00,Wolverhampton Wanderers FC,Manchester City FC,3,2
6,264533,Premier League,2019-12-28 15:00:00,Newcastle United FC,Everton FC,1,2
7,264538,Premier League,2019-12-28 15:00:00,Southampton FC,Crystal Palace FC,1,1
8,264535,Premier League,2019-12-28 17:30:00,Norwich City FC,Tottenham Hotspur FC,2,2
9,264537,Premier League,2019-12-28 17:30:00,West Ham United FC,Leicester City FC,1,2


#                                             THANK YOU

In [102]:
#Football_data['Date'] = Football_data['utcDate'].dt.date
#Football_data['Time'] = [datetime.datetime.time(d) for d in Football_data['utcDate']] 
###Football_data1=Football_data[['Date','Time','utcDate','awayTeam.id', 'homeTeam.id', 'competition.name', 'awayTeam.name', 'homeTeam.name','score.fullTime.awayTeam','score.fullTime.homeTeam']]


In [103]:
#Football_data=Football_data[['id','utcDate', 'competition.name', 'awayTeam.name', 'homeTeam.name','score.fullTime.awayTeam','score.fullTime.homeTeam']]